In [141]:
import sys
import numpy as np
import pandas as pd
import copy
import random
import math
from scipy import stats
from scipy.stats import rankdata
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics, preprocessing
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
np.random.seed(1)

In [142]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital', 'occupation', 'relationship', 'race', 'gender', 'capgain', 'caploss', 'hours', 'country', 'income']
df_train = pd.read_csv('adult.data', names=cols, sep=",")
df_test = pd.read_csv('adult.test', names=cols, sep=",")

In [143]:
 def preprocess(df):
    df.isin(['?']).sum(axis=0)

    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace('?',np.nan)
    df['workclass'] = df['workclass'].replace('?',np.nan)
    df['occupation'] = df['occupation'].replace('?',np.nan)

    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
            
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
    df['gender'] = df['gender'].map({'Male': 1, 'Female': 0}).astype(int)
    df['workclass'] = df['workclass'].map({'State-gov': 0, 'Self-emp-not-inc': 1, 'Private': 2, 'Federal-gov': 3, 'Local-gov': 4, '?': 5,
                                           'Self-emp-inc': 6, 'Without-pay': 7, 'Never-worked': 8}).astype(int)
    df['education'] = df['education'].map({'Bachelors': 0, 'HS-grad': 1, '11th': 2, 'Masters': 3, '9th': 4, 
                                           'Some-college': 5, 'Assoc-acdm': 6, 'Assoc-voc': 7, '7th-8th': 8, 'Doctorate': 9, 
                                           'Prof-school': 10, '5th-6th': 11, '10th': 12, '1st-4th': 13, 'Preschool': 14, '12th': 15}).astype(int)
    df['marital'] = df['marital'].map({'Never-married': 0, 'Married-civ-spouse': 1, 'Divorced': 2, 'Married-spouse-absent': 3, 
                                                     'Separated': 4, 'Married-AF-spouse': 5, 'Widowed': 6}).astype(int)
    df['occupation'] = df['occupation'].map({'Adm-clerical': 0, 'Exec-managerial': 1, 'Handlers-cleaners': 2, 
                                             'Prof-specialty': 3, 'Other-service': 4, 'Sales': 5, 'Craft-repair': 6, 'Transport-moving': 7, 'Farming-fishing': 8, 
                                             'Machine-op-inspct': 9, 'Tech-support': 10, '?': 11, 'Protective-serv': 12, 'Armed-Forces': 13, 'Priv-house-serv': 14}).astype(int)
    df['relationship'] = df['relationship'].map({'Not-in-family': 0, 'Husband': 1, 'Wife': 2, 
                                                 'Own-child': 3, 'Unmarried': 4, 'Other-relative': 5}).astype(int)
    df['race'] = df['race'].map({'White': 0, 'Black': 1, 'Asian-Pac-Islander': 2, 'Amer-Indian-Eskimo': 3, 'Other': 4}).astype(int)
    df['country'] = df['country'].map({'United-States': 0, 'Cuba': 1, 'Jamaica': 2, 'India': 3, '?': 4, 'Mexico': 5, 'South': 6, 'Puerto-Rico': 7, 
                                       'Honduras': 8, 'England': 9, 'Canada': 10, 'Germany': 11, 'Iran': 12, 'Philippines': 13, 'Italy': 14, 
                                       'Poland': 15, 'Columbia': 16, 'Cambodia': 17, 'Thailand': 18, 'Ecuador': 19, 'Laos': 20, 'Taiwan': 21, 
                                       'Haiti': 22, 'Portugal': 23, 'Dominican-Republic': 24, 'El-Salvador': 25, 'France': 26, 'Guatemala': 27, 
                                       'China': 28, 'Japan': 29, 'Yugoslavia': 30, 'Peru': 31, 'Outlying-US(Guam-USVI-etc)': 32, 'Scotland': 33,
                                       'Trinadad&Tobago': 34, 'Greece': 35, 'Nicaragua': 36, 'Vietnam': 37, 'Hong': 38, 'Ireland': 39, 'Hungary': 40, 
                                       'Holand-Netherlands': 41}).astype(int)
    
    
    labels = df['age']
    proc = []
    for v in labels:
            if v <= 30:
                proc.append(1)
            elif v <= 40:
                proc.append(2)
            elif v <= 50:
                proc.append(3)
            else:
                proc.append(4)
    df['age']=proc 
    
    labels = df['hours']
    proc=[]
    for v in labels:
        if v<=25:
            proc.append(1)
        elif v<=41:
            proc.append(2)
        elif v<=55:
            proc.append(3)
        else:
            proc.append(4)
    df['hours']=proc
    
    df = df.drop(['fnlwgt', 'education.num', 'capgain', 'caploss', 'country'], axis = 1, inplace = True) 

**One-hot encoding**

In [144]:
 def one_hot_encode(df):
    df.isin(['?']).sum(axis=0)

    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace('?',np.nan)
    df['workclass'] = df['workclass'].replace('?',np.nan)
    df['occupation'] = df['occupation'].replace('?',np.nan)

    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
    df = pd.concat([df, pd.get_dummies(df['gender'], prefix='gender')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['race'], prefix='race')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['marital'], prefix='marital')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['workclass'], prefix='workclass')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['relationship'], prefix='relationship')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['occupation'], prefix='occupation')],axis=1)

    df = df.drop(columns=['workclass', 'gender', 'fnlwgt', 'education', 'occupation', \
                      'relationship', 'marital', 'race', 'country', 'capgain', \
                      'caploss'])
    return df

In [145]:
# not one-hot (for randomforestclassifier and such)
# preprocess(df_train)
# preprocess(df_test)

# one-hot encoding (for regression mdoels)
df_train = one_hot_encode(df_train)
df_test = one_hot_encode(df_test)

In [146]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

**Protected, privileged**

In [147]:
# protected: 'gender'=0
# privileged: 'gender'=1

**Parametric Model**

In [148]:
X_train = df_train.drop(columns='income')
y_train = df_train['income']

X_test = df_test.drop(columns='income')
y_test = df_test['income']

# size=10000
# X_train = X_train[0:size]
# y_train = y_train[0:size]

# X_test = X_test[0:size]
# y_test = y_test[0:size]

X_test_orig = copy.deepcopy(X_test)

# Scale data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# X_train = preprocessing.scale(X_train)
# X_test = preprocessing.scale(X_test)

# Logistic regression
clf = LogisticRegression(random_state=0)

clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
num_params = len(clf.coef_.transpose()) + 1 #weights and intercept
# clf.classes_
# clf.coef_

**Compute statistical parity difference**

In [149]:
def computeFairness(y_pred, X_test): 
    protected_idx = X_test[X_test['gender_Female']==1].index
    numProtected = len(protected_idx)
    privileged_idx = X_test[X_test['gender_Male']==1].index
    numPrivileged = len(privileged_idx)
    
    p_protected = 0
    for i in range(len(protected_idx)):
        p_protected += y_pred[protected_idx[i]][1]
    p_protected /= len(protected_idx)
    
    p_privileged = 0
    for i in range(len(privileged_idx)):
        p_privileged += y_pred[privileged_idx[i]][1]
    p_privileged /= len(privileged_idx)
    
    spd = p_protected - p_privileged
    return spd

**Influence of points computed using ground truth**

In [150]:
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
spd_0 = computeFairness(y_pred, X_test_orig)

delta_spd = []
for i in range(len(X_train)):
    X_removed = np.delete(X_train, i, 0)
    y_removed = y_train.drop(index=i, inplace=False)
    clf.fit(X_removed, y_removed)
    y_pred = clf.predict_proba(X_test)
    delta_spd_i = computeFairness(y_pred, X_test_orig) - spd_0
    delta_spd.append(delta_spd_i)
print(delta_spd)

with open('delta_spd_ground_truth_v0.txt', 'w') as filehandle:
    for listitem in delta_spd:
        filehandle.write('%s\n' % listitem)

[-2.9897359587749417e-06, -1.7448931566160786e-05, 2.953688038737079e-06, -4.243348493304655e-06, 7.820858306670231e-05, 9.024216627701032e-05, 3.051729909109291e-06, 2.9591824772079445e-05, -8.542187826243719e-05, 1.3732451290215941e-05, 1.5089882037244706e-05, 2.5279052907889454e-05, 2.9057901615447967e-06, -5.057105453648569e-06, 3.6948667095915155e-06, 5.9180397615210545e-06, 2.0688480339980941e-07, -1.1882299095267168e-05, -8.25465472916731e-05, 6.7216688959259585e-06, 3.771799030105605e-06, -2.6963830247750398e-06, -7.427121266195824e-06, 7.502395309111343e-06, 1.3514899243671774e-05, 5.427764008275604e-06, -1.6537867079113866e-05, -1.0231531495374924e-05, -3.3620170047932163e-06, 6.29840397586845e-07, -6.159350407264341e-06, -4.247611410379548e-06, 3.741340644403124e-06, 1.7047720684559842e-06, 5.344466122880753e-06, 3.982882768654994e-05, -2.9360272215395167e-05, -7.837136737631756e-07, -3.39966463104302e-05, -2.6616494148368197e-05, 5.47739088882504e-06, 2.7026372342864313e-06

In [151]:
clf.fit(X_train, y_train)

LogisticRegression(random_state=0)

**Loss function** (Log loss for logistic regression)

In [152]:
def logistic_loss(y_true, y_pred):
    loss = 0
    for i in range(len(y_true)):
        loss += - y_true[i] * math.log(y_pred[i][1]) - (1 - y_true[i]) * math.log(y_pred[i][0])
    loss /= len(y_true)
    return loss

print(logistic_loss(y_test, y_pred))

0.36097107394720573


**First-order derivative of loss function at z with respect to model parameters**

In [153]:
def del_L_del_theta_i(num_params, y_true, x, y_pred):
    del_L_del_theta = np.ones((num_params, 1)) * ((1 - y_true) * y_pred[1] - y_true * y_pred[0])
    for j in range(1, num_params):
            del_L_del_theta[j] *=  x[j-1]
    return del_L_del_theta
# value to be multiplied by a factor of 1/n

**Second-order partial derivative of loss function with respect to model parameters**

In [154]:
def hessian_one_point(num_params, x, y_pred):
#     print(y_pred[0] * y_pred[1])
    H = np.ones((num_params, num_params)) * (y_pred[0] * y_pred[1])
    for i in range(1, num_params):
        for j in range(i+1):
            if j == 0:
                H[i][j] *= x[i-1]
            else:
                H[i][j] *= x[i-1] * x[j-1] 
    i_lower = np.tril_indices(num_params, -1)
    H.T[i_lower] = H[i_lower]     
    return H
# value to be multiplied by a factor of 1/n

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [155]:
def del_f_del_theta_i(num_params, x, y_pred):
    del_f_del_theta = np.ones((num_params, 1)) * (y_pred[0] * y_pred[1])
    for j in range(1, num_params):
            del_f_del_theta[j] *=  x[j-1]
    return del_f_del_theta

**Computing $v=\nabla($Statistical parity difference$)$**

In [156]:
# Return v = del(SPD)/del(theta)
del_f_protected = np.zeros((num_params, 1))
del_f_privileged = np.zeros((num_params, 1))
numProtected = X_test_orig['gender_Female'].sum()
numPrivileged = X_test_orig['gender_Male'].sum()
for i in range(len(X_test)):
    y_pred = clf.predict_proba(np.reshape(X_test[i], (1, num_params-1)))
    del_f_i = del_f_del_theta_i(num_params, X_test[i], y_pred[0])
    if X_test_orig.iloc[i]['gender_Male'] == 1: #privileged
        del_f_privileged = np.add(del_f_privileged, del_f_i)
    elif X_test_orig.iloc[i]['gender_Female'] == 1:
        del_f_protected = np.add(del_f_protected, del_f_i)
del_f_privileged /= numPrivileged
del_f_protected /= numProtected
v = np.subtract(del_f_protected, del_f_privileged)
print(v.transpose())

[[-0.07592556 -0.02845483 -0.00188227 -0.048868    0.18897018 -0.18897018
   0.00432292  0.00389831  0.02087455  0.00332219 -0.02144056  0.05088862
   0.01123877 -0.10314517  0.00858797  0.05175203  0.01726284  0.03296542
  -0.00357517  0.01094144  0.01323908 -0.0188173  -0.02027118  0.00537222
   0.00162495 -0.16729359  0.05124135  0.01690447  0.03085331  0.05147186
   0.14380155  0.04574062 -0.00243153 -0.04495888 -0.00261241 -0.00736998
   0.00239309 -0.00383049  0.02655014  0.00646609  0.0154396  -0.01892167
  -0.00983248  0.00367317 -0.02545955]]


**Stochastic estimation of Hessian vector product: $H_{\theta}^{-1}v = H_{\theta}^{-1}\nabla_{\theta}f(z, \theta) = v + [I - \nabla_{\theta}^2L(z_{s_j}, \theta^*)]H_{\theta}^{-1}v$**

In [157]:
# Uniformly sample t points from training data 
size = len(X_train)
sample = random.sample(range(len(X_train)), size)

hinv_v = copy.deepcopy(v)
for idx in range(size):
    i = sample[idx]
    y_pred = clf.predict_proba(np.reshape(X_train[i], (1, num_params-1)))
    hessian_i = hessian_one_point(num_params, X_train[i], y_pred[0])/len(X_train)
    hinv_v = np.matmul(np.subtract(np.identity(num_params), hessian_i), hinv_v)
#     print(hessian_i)
    hinv_v = np.add(hinv_v, v)
print(hinv_v.transpose())

[[-1675.24727502  -584.28537019   -40.48244086 -1118.30419631
   4723.01619466 -4723.01619466    92.33692354    91.77903758
    445.58989614    70.28990257  -461.71752177  1143.62338324
    249.99603063 -2234.45016525   173.69170689  1065.71917384
    374.26114072   792.14722127  -117.24847275   257.38151474
    270.65085793  -392.92291856  -427.16036605   130.45179412
     35.75335578 -3829.4807922   1051.2857437    398.08345869
    664.83719796  1199.47282189  3573.42770797  1052.65722434
    -71.3016037  -1018.76343461   -21.37745056  -159.88368427
     15.44297562  -100.81947071   590.59265953   150.69702374
    328.15093912  -438.44790306  -212.83977681    92.01064745
   -578.99442072]]


**Influence of points computed using Hessian vector product**

In [164]:
infs = []
clf.fit(X_train, y_train)
for i in range(len(X_train)):
    y_pred = clf.predict_proba(np.reshape(X_train[i], (1, num_params-1)))
    del_L_del_theta = del_L_del_theta_i(num_params, y_train[i], X_train[i], y_pred[0])/len(X_train)
    inf = -np.dot(del_L_del_theta.transpose(), hinv_v)
    inf /= -1/len(X_train)
    infs.append(inf[0][0].tolist())

In [165]:
ifs = pd.DataFrame(infs, columns=["Values"])

In [168]:
stats.spearmanr(delta_spd, infs)
# stats.pearsonr(delta_spd, infs)

SpearmanrResult(correlation=0.9385733472433024, pvalue=0.0)

**Inverse of Hessian computed using pseudo-inverse, spd computed**

In [44]:
delta_spd_pinv = []
for i in range(len(X_train)):
    y_pred = clf.predict_proba(np.reshape(X_train[i], (1, num_params-1)))
    del_L_del_theta = del_L_del_theta_i(num_params, y_train[i], X_train[i], y_pred[0])/len(X_train)
    updated_model_params = np.matmul(del_L_del_theta.transpose(), hinv_v)/len(X_train)
    clf.coef_ = np.add(clf.coef_, updated_model_params)
    y_pred = clf.predict_proba(X_test)
    delta_spd_i = spd_0 - computeFairness(y_pred, X_test_orig)
    delta_spd_pinv.append(delta_spd_i)

# with open('delta_spd_pinv_v1.txt', 'w') as filehandle:
#     for listitem in delta_spd_pinv:
#         filehandle.write('%s\n' % listitem)

In [45]:
# gt_rank = rankdata(delta_spd).astype(int)
pinv_rank = rankdata(delta_spd_pinv).astype(int)

**Comparing true updated parameters with those approximated using influence functions**

In [18]:
for i in range(10):
    X_removed = np.delete(X_train, i, 0)
    y_removed = y_train.drop(index=i, inplace=False)
    clf.fit(X_removed, y_removed)
    print("Ground truth updated parameters")
    print(clf.coef_)

    y_pred = clf.predict_proba(np.reshape(X_train[i], (1, num_params-1)))
    del_L_del_theta = del_L_del_theta_i(num_params, y_train[i], X_train[i], y_pred[0])
    updated_model_params = np.matmul(del_L_del_theta.transpose(), hinv_v)/len(X_train)
    clf.coef_ = np.add(clf.coef_, updated_model_params)
    print("Approximated parameters")
    print(clf.coef_)

Ground truth updated parameters
[[ 0.39056678  0.75705478  0.36234289 -0.20205895  0.20205895 -0.04144044
  -0.00763714 -0.00111484 -0.05638538  0.03039132 -0.20594506  0.04942333
   0.67148257 -0.07327921 -0.48127384 -0.11482732 -0.0729876   0.09608792
  -0.02771863  0.04542855  0.06303426 -0.09615068 -0.05725656 -0.15375514
  -0.03088295  0.15258774 -0.08591606 -0.3029528   0.04571113  0.26863218
  -0.01759744 -0.01511476  0.00294718  0.26165246 -0.17955955 -0.15909311
  -0.08674547 -0.30169847 -0.17979714  0.18531302  0.07166639  0.08082351
   0.09364591 -0.03407036]]
Approximated parameters
[[ 3.92051008e-01  7.58539007e-01  3.63827113e-01 -2.00574724e-01
   2.03543173e-01 -3.99562130e-02 -6.15291341e-03  3.69388969e-04
  -5.49011607e-02  3.18755445e-02 -2.04460831e-01  5.09075533e-02
   6.72966795e-01 -7.17949862e-02 -4.79789615e-01 -1.13343092e-01
  -7.15033735e-02  9.75721413e-02 -2.62344081e-02  4.69127695e-02
   6.45184874e-02 -9.46664585e-02 -5.57723311e-02 -1.52270915e-01
  

  -0.01205529 -0.13983945]]
Ground truth updated parameters
[[ 0.39054457  0.75696883  0.36234988 -0.20199522  0.20199522 -0.04145605
  -0.00763212 -0.00110534 -0.05637707  0.03038318 -0.2059432   0.04942047
   0.67155936 -0.07329337 -0.48135845 -0.11481176 -0.07299047  0.09613778
  -0.02769306  0.04542104  0.06307417 -0.09614294 -0.05735942 -0.153785
  -0.03096425  0.15256933 -0.0859008  -0.3028014   0.04572364  0.2685746
  -0.01770199 -0.01512055  0.00298237  0.26154909 -0.17954454 -0.15907529
  -0.08675646 -0.30168507 -0.17983861  0.18541132  0.07168053  0.08086355
   0.09365822 -0.03406123]]
Approximated parameters
[[ 0.42324239  0.78966666  0.3950477  -0.1692974   0.23469305 -0.00875822
   0.02506571  0.03159249 -0.02367925  0.06308101 -0.17324537  0.0821183
   0.70425718 -0.04059554 -0.44866062 -0.08211394 -0.04029265  0.1288356
   0.00500477  0.07811887  0.095772   -0.06344511 -0.0246616  -0.12108717
   0.00173357  0.18526716 -0.05320298 -0.27010357  0.07842147  0.30127242
   0.

In [20]:
delta_spd = pd.read_csv('delta_spd_ground_truth.txt', names=["Values"], sep=",")
# delta_spd_pinv = pd.read_csv('delta_spd_pinv.txt', names=["Values"], sep=",")

gt_rank = rankdata(delta_spd).astype(int)
pinv_rank = rankdata(delta_spd_pinv).astype(int)

stats.spearmanr(delta_spd, delta_spd_pinv)
# stats.pearsonr(delta_spd, delta_spd_pinv)
# stats.pearsonr(gt_rank, pinv_rank)

**Full Hessian matrix of loss function with respect to model parameters**

In [25]:
clf.fit(X_train, y_train)
def compute_hessian(num_params, X_train, y_train):
    H = np.zeros((num_params, num_params))
    for i in range(len(X_train)):
        y_pred = clf.predict_proba(np.reshape(X_train[i], (1, num_params-1)))
        H = np.add(H, hessian_one_point(num_params, X_train[i], y_pred[0])/len(X_train))
    H /= len(X_train)
    return H

hxx = compute_hessian(num_params, X_train, y_train)
hxx

**Challenges: (1) Hessian is ill-conditioned; (2) Inverting Hessian is computationally expensive.**
Solutions: (1) Resort to pseudo-inverse; (2) Use Hessian vector products

In [113]:
# Traditional way of computing hinv produces results that are prone 
# to numerical errors because hxx is ill-conditioned
# If the condition number is very large, then the matrix is said to be ill-conditioned. 
# Practically, such a matrix is almost singular, and the computation of its inverse, 
# or solution of a linear system of equations is prone to large numerical errors.
# Ax = b; Small change in elements of b effects huge change in the solution x. 
# Condition number >= 1. (=1 for identity matrix I)

# hinv = np.linalg.inv(hxx)

# We, therefore, compute pseudo-inverse
hpinv = np.linalg.pinv(hxx)


i:  0
Maximum change:  1.2736311704568954
Minimum change:  -0.255102107041807
i:  1
Maximum change:  0.8288609844597572
Minimum change:  -1.1969165625533587
i:  2
Maximum change:  2.4088418062740358
Minimum change:  -0.568077483922055
i:  3
Maximum change:  3.2829238158492258
Minimum change:  -1.5078677722105867
i:  4
Maximum change:  3.8507609025641623
Minimum change:  -2.1970274212329213
i:  5
Maximum change:  1.4531599084270606
Minimum change:  -1.159364016420451
i:  6
Maximum change:  4.623801470250959
Minimum change:  -2.061683589113863
